#  AI Demand Forecasting & Dynamic Replenishment Agent
## Production-Ready ML System for Inventory Optimization

This notebook demonstrates a complete AI agent for demand forecasting using Qwen2.5 models.

**Key Features:**
- Continuous learning from historical sales data
- Multi-factor analysis (seasonality, trends, external factors)
- Dynamic reorder point calculation
- Interactive web dashboard
- AI-powered insights using Qwen2.5-7B-Instruct

**Learning Objectives:**
- Generate synthetic sales data with realistic patterns
- Engineer features for time series forecasting
- Train gradient boosting models for demand prediction
- Calculate dynamic reorder recommendations
- Deploy interactive dashboard for business users

In [ ]:
# %% Setup
import sys
sys.path.insert(0, '..')

from src.data_generator import SalesDataGenerator
from src.forecasting_agent import DemandForecastingAgent
from src.model_loader import ModelLoader
from src.dashboard import ForecastingDashboard

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (16, 10)
%matplotlib inline

print(" Imports successful!")
print(f" Pandas: {pd.__version__}")
print(f" NumPy: {np.__version__}")

In [ ]:
# %% Generate Synthetic Sales Data
# Generate 2 years of sales data with realistic patterns:
# - Trends, seasonality, weekly patterns
# - Holiday effects
# - Random variations and stockouts

# Initialize data generator
generator = SalesDataGenerator(seed=42)

print(" Generating synthetic sales data...")
sales_df = generator.generate_sales_data(
    start_date='2022-01-01',
    periods=730,  # 2 years
    n_skus=10     # 10 products
)

# Generate external factors
print("  Generating external factors...")
external_df = generator.generate_external_factors(sales_df)

# Generate inventory status
print(" Generating inventory data...")
inventory_df = generator.generate_inventory_data(sales_df)

print(f"\n Generated {len(sales_df):,} sales records")
print(f" SKUs: {sales_df['sku_id'].nunique()}")
print(f" Date range: {sales_df['date'].min()} to {sales_df['date'].max()}")

display(sales_df.head())

In [ ]:
# %% Visualize Historical Sales Data
# Plot sales trends for first SKU with moving averages

# Plot sales for first SKU
sku = 'SKU_001'
sku_data = sales_df[sales_df['sku_id'] == sku].copy()
sku_data['date'] = pd.to_datetime(sku_data['date'])

plt.figure(figsize=(15, 6))
plt.plot(sku_data['date'], sku_data['sales'], alpha=0.6, label='Daily Sales', linewidth=1)
plt.plot(sku_data['date'], sku_data['sales'].rolling(30).mean(), 
         color='red', linewidth=2.5, label='30-Day Moving Average')
plt.title(f'Historical Sales - {sku}', fontsize=14, fontweight='bold')
plt.xlabel('Date', fontsize=12, fontweight='bold')
plt.ylabel('Units Sold', fontsize=12, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(" Historical sales visualization complete!")

In [ ]:
# %% Category Distribution Analysis
# Analyze sales distribution across product categories

# Category sales breakdown
category_sales = sales_df.groupby('category')['sales'].sum().sort_values(ascending=False)

plt.figure(figsize=(10, 6))
category_sales.plot(kind='bar', color='steelblue', alpha=0.8)
plt.title('Total Sales by Category', fontsize=14, fontweight='bold')
plt.xlabel('Category', fontsize=12, fontweight='bold')
plt.ylabel('Total Units Sold', fontsize=12, fontweight='bold')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

print("\n Category Summary:")
print(category_sales)
print("\n Category analysis complete!")

In [ ]:
# %% Initialize Forecasting Agent
# Create AI agent with machine learning capabilities

agent = DemandForecastingAgent()
print(" Demand Forecasting Agent initialized")
print("\n Agent Capabilities:")
print("   • Feature engineering (lag, rolling statistics)")
print("   • Gradient Boosting Regressor")
print("   • Continuous learning from new data")
print("   • Dynamic reorder calculations")
print("   • Forecast accuracy tracking (MAPE, R²)")

In [ ]:
# %% Train Forecasting Models
# Train ML models for first 3 SKUs
# Models learn patterns from historical data

skus = sales_df['sku_id'].unique()[:3]

print(" Training forecasting models...\n")
for i, sku in enumerate(skus, 1):
    print(f"[{i}/{len(skus)}] Training model for {sku}...", end=" ")
    model, metrics = agent.train_model(sku, sales_df, external_df)
    print(f" MAPE: {metrics['mape']:.2f}%, R²: {metrics['test_score']:.2%}")

print("\n Model training complete!")

# Show model performance
performance_df = agent.get_model_performance()
print("\n Model Performance Summary:\n")
display(performance_df)

In [ ]:
# %% Generate 30-Day Forecast
# Generate demand predictions for next 30 days
# Includes confidence intervals for uncertainty quantification

sku = 'SKU_001'
future_dates = pd.date_range(start=pd.Timestamp.now(), periods=30, freq='D')

print(f" Generating 30-day forecast for {sku}...")
forecast_df = agent.predict_demand(sku, future_dates, external_df)

print("\n Forecast generated successfully!")
print(f"   Avg daily demand: {forecast_df['predicted_demand'].mean():.1f} units")
print(f"   Peak demand: {forecast_df['predicted_demand'].max()} units")
print(f"   Total 30-day demand: {forecast_df['predicted_demand'].sum():,} units")

print("\n Forecast Summary (First 10 Days):\n")
display(forecast_df.head(10))

In [ ]:
# %% Visualize Forecast with Inventory Levels
# Plot forecast with confidence intervals and inventory thresholds

plt.figure(figsize=(15, 6))

# Plot forecast
plt.plot(forecast_df['date'], forecast_df['predicted_demand'], 
         'b-', linewidth=2.5, label='Predicted Demand', marker='o', markersize=5)

# Confidence interval
plt.fill_between(forecast_df['date'], 
                  forecast_df['lower_bound'], 
                  forecast_df['upper_bound'],
                  alpha=0.3, label='95% Confidence Interval')

# Inventory levels
inv_info = inventory_df[inventory_df['sku_id'] == sku].iloc[0]
plt.axhline(y=inv_info['current_stock'], color='green', 
            linestyle='--', linewidth=2, label=f"Current Stock ({inv_info['current_stock']} units)")
plt.axhline(y=inv_info['reorder_point'], color='red', 
            linestyle=':', linewidth=2, label=f"Reorder Point ({inv_info['reorder_point']} units)")

plt.title(f'30-Day Demand Forecast - {sku}', fontsize=14, fontweight='bold')
plt.xlabel('Date', fontsize=12, fontweight='bold')
plt.ylabel('Units', fontsize=12, fontweight='bold')
plt.legend(fontsize=11, loc='best')
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print(" Forecast visualization complete!")

In [ ]:
# %% Calculate Dynamic Reorder Recommendation
# Calculate optimal reorder point and quantity
# Based on forecast, lead time, and safety stock

reorder_info = agent.calculate_dynamic_reorder(
    sku,
    forecast_df,
    inv_info['current_stock'],
    inv_info['lead_time_days']
)

print(f"\n{'='*60}")
print(f" REORDER RECOMMENDATION - {sku}")
print(f"{'='*60}")

print(f"\n CURRENT STATUS:")
print(f"   Current Stock: {reorder_info['current_stock']:,} units")
print(f"   Reorder Point: {reorder_info['reorder_point']:,} units")
print(f"   Safety Stock: {reorder_info['safety_stock']:,} units")
print(f"   Lead Time: {inv_info['lead_time_days']} days")

print(f"\n FORECAST INSIGHTS:")
print(f"   Expected Lead Time Demand: {reorder_info['lead_time_demand']:,} units")
print(f"   Days Until Stockout: {reorder_info['days_until_stockout']} days")
print(f"   Forecast Avg Daily Demand: {forecast_df['predicted_demand'].mean():.1f} units")

print(f"\n RECOMMENDATION:")
print(f"   Needs Reorder: {'YES ⚠️' if reorder_info['needs_reorder'] else 'NO ✓'}")
print(f"   Urgency Level: {reorder_info['urgency']}")
print(f"   Recommended Order Quantity: {reorder_info['reorder_quantity']:,} units")

if reorder_info['needs_reorder']:
    print(f"\n    ACTION REQUIRED: Place order immediately to avoid stockout!")

print(f"\n{'='*60}\n")

In [ ]:
# %% Optional: AI-Powered Analysis with Qwen2.5
# OPTIONAL: Generate AI insights using Qwen2.5 model
# Skip this cell if you don't have a local model or want to avoid downloading
#
# Requirements:
# - Local model running at http://127.0.0.1:8080, OR
# - ~5GB disk space and 8GB+ RAM to download model

# Option 1: Use LOCAL model (recommended)
model_loader = ModelLoader(
    model_name='qwen2.5-7b',
    use_local=True,
    local_endpoint='http://127.0.0.1:8080'
)
model_loader.load_model()

# Option 2: Download model (uncomment if you want to download)
# model_loader = ModelLoader(model_name='qwen2.5-7b', device='cpu')
# model_loader.load_model()

# Prepare analysis data
sku_history = sales_df[sales_df['sku_id'] == sku].tail(90)
historical_data = {
    'avg_sales': sku_history['sales'].mean(),
    'trend': 'increasing' if sku_history['sales'].iloc[-1] > sku_history['sales'].iloc[0] else 'decreasing',
    'seasonality': 'moderate',
    'stockout_rate': (sales_df['stockout'].sum() / len(sales_df)) * 100,
    'current_stock': inv_info['current_stock'],
    'reorder_point': inv_info['reorder_point'],
    'lead_time': inv_info['lead_time_days']
}

forecast_data = {
    'predicted_avg': forecast_df['predicted_demand'].mean(),
    'peak_demand': forecast_df['predicted_demand'].max(),
    'peak_day': forecast_df['predicted_demand'].idxmax() + 1,
    'confidence': 95
}

external_factors = {
    'economic_index': external_df['economic_index'].mean() if 'economic_index' in external_df.columns else 100,
    'seasonal': 'current',
    'promotion': 'active' if external_df['promotion_active'].sum() > 0 else 'none'
}

# Generate AI analysis
print(" Generating AI-powered analysis...\n")
print("This may take 10-30 seconds depending on your setup...\n")

try:
    analysis = model_loader.analyze_forecast(
        historical_data,
        forecast_data,
        external_factors
    )
    
    print("="*70)
    print(" AI ANALYSIS")
    print("="*70)
    print(analysis)
    print("="*70)
    
    print("\n AI analysis complete!")
    
except Exception as e:
    print(f" AI analysis failed: {e}")
    print("Continuing without AI insights...")

In [ ]:
# %% Launch Interactive Dashboard
# Start web-based dashboard for interactive analysis
# Access at http://127.0.0.1:8050
#
# Features:
# - SKU selection dropdown
# - Adjustable forecast horizon (7-90 days)
# - Real-time visualizations
# - Reorder recommendations
# - Model performance metrics

dashboard = ForecastingDashboard(agent, sales_df, external_df, inventory_df)

print("\n" + "="*70)
print(" LAUNCHING INTERACTIVE DASHBOARD")
print("="*70)
print("\n URL: http://127.0.0.1:8050")
print("\n Dashboard Features:")
print("   • Select SKU and forecast horizon")
print("   • View demand predictions with confidence intervals")
print("   • Get dynamic reorder recommendations")
print("   • Analyze historical sales patterns")
print("   • Track model performance metrics")
print("   • Generate AI-powered insights")
print("\n  Note: Dashboard runs in blocking mode")
print("   Press Ctrl+C in terminal to stop the server")
print("="*70 + "\n")

dashboard.run(host='127.0.0.1', port=8050, debug=True)

In [ ]:
# %% Save Results
# Export data and forecasts to CSV files

print(" Saving data files...")

sales_df.to_csv('../data/sales_data.csv', index=False)
external_df.to_csv('../data/external_factors.csv', index=False)
inventory_df.to_csv('../data/inventory_status.csv', index=False)
forecast_df.to_csv('../data/forecast_results.csv', index=False)

print("\n Files saved successfully:")
print("   • data/sales_data.csv")
print("   • data/external_factors.csv")
print("   • data/inventory_status.csv")
print("   • data/forecast_results.csv")

print("\n Analysis complete!")